In [7]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import pymongo
import time

In [8]:
# Set Executable Path & Initialize Chrome Browser

from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [9]:
url =  "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(1)

In [10]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [11]:
# 1 NASA Mars News
news_title = soup.find("div", class_="list_text").get_text()
print(news_title)

April  1, 2021NASA's InSight Detects Two Sizable Quakes on MarsThe magnitude 3.3 and 3.1 temblors originated in a region called Cerberus Fossae, further supporting the idea that this location is seismically active.


In [12]:
news_paragraph = soup.find("div", class_="article_teaser_body").get_text()
print(news_paragraph)

The magnitude 3.3 and 3.1 temblors originated in a region called Cerberus Fossae, further supporting the idea that this location is seismically active.


In [13]:
#JPL Mars Space Images - Featured Image
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)
url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(url)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [14]:
# Scrape page into Soup
html = browser.html
image_soup = bs(html, "html.parser")

In [15]:
# 2 Search for image source
# Use splinter to navigate the site and find the image url for the current Featured Mars Image
relative_img_path = image_soup.find_all('img')[1]["src"]

results = image_soup.find_all("img", class_="floating_text_area")

for result in results:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
        link = result.find('a')
        href = link['href']
url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
featured_img_url = url + relative_img_path

print(featured_img_url)


https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


In [30]:
# 3 Mars Facts
# use Pandas to scrape the table containing facts about the planet 
fact_tables = pd.read_html('https://space-facts.com/mars/')
df = fact_tables[0]
df = df.iloc[0:]
df.columns=['Description', 'Values']
df.set_index('Description', inplace=True)
df

,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
#Use Pandas to convert the data to a HTML table string.
#mars_facts_table = [df.to_html(classes='data table table-borderless', index=False, header=False, border=0)]
mars_facts_table = df.to_html()
mars_facts_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Description</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astrono

In [18]:
#save the table directly to a file.
df.to_html('table.html')
print(df)

            Description                         Values
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [28]:
# 4 Mars Hemispheres
# Visit the USGS Astrogeology and obtain high resolution images for each of Mar's hemispheres.
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
base_url = "https://astrogeology.usgs.gov"
browser.visit(url)
time.sleep(1)

html = browser.html
soup = bs(html, 'html.parser')

hemi_names = []
hemi_urls = []

results = soup.find_all('div', class_="item")
dict_list = []

for i in results:
        hemispheres_titles = i.find('h3').text
        
        target_url = base_url + i.find('a')['href']
        browser.visit(target_url)
        time.sleep(1)
        
        final_img_html = browser.html
        soup = bs( final_img_html, 'html.parser')
        hemispheres_urls = base_url + soup.find('img', class_='wide-image')['src']
        
        
        entity_dict = {"title":hemispheres_titles, "img_url":hemispheres_urls}
        dict_list.append(entity_dict)

In [29]:
dict_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]